In [15]:
%matplotlib inline
from fenparse import *
from pybrain.datasets import ClassificationDataSet
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.tools.shortcuts import buildNetwork
from pybrain.structure.modules import TanhLayer
import numpy as np
import random
from sklearn.metrics import confusion_matrix

In [5]:

NUM_LEGAL = 5
NUM_ILLEGAL = 5
with open("../data/2015-12-01-fen_sample") as f:
    content = [" ".join(line.split(" ")[:2]) for line in f.readlines()]

In [6]:
legal = []
illegal = []
for fen in content[:5000]:
    iboard, to_move = fen2iboard(fen)
    #rotate if necessary
    iboard = rotate_iboard(iboard) if to_move == 'b' else iboard
    #generate n legal moves
    try :
        legal += [move2keras(iboard, get_legal_move_generator(iboard).next()) for i in range(0, NUM_LEGAL)]
    except StopIteration:
        pass
    #generate m illegal moves
    for i in range(0, NUM_ILLEGAL):
        next_move = gen_illegal_moves(iboard, get_legal_move_generator(iboard)).next()
        if (type(next_move) == tuple):
            illegal.append(move2keras(iboard, next_move))

#prettify_keras(illegal[4])


In [7]:
legal_train_sample = random.sample(legal, 1000)
legal_test_sample = random.sample(legal, 1000)
illegal_train_sample = random.sample(illegal, 1000)
illegal_test_sample = random.sample(illegal, 1000)
X_train = (legal_train_sample + illegal_train_sample)
y_train = [[1] for i in range(0,len(legal_train_sample))] + [[0] for i in range(0,len(illegal_train_sample))]
X_test = legal_test_sample + illegal_test_sample
y_test = [[1] for i in range(0,len(legal_test_sample))] + [[0] for i in range(0,len(illegal_test_sample))]
assert len(X_train) == len(X_test)

In [10]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_train)
y_test = np.array(y_test)

X_train[:5]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [32]:
#train a simple neural network on 2000 instances

# build dataset
train_ds = ClassificationDataSet(1536, nb_classes=2, class_labels=["Illegal, Legal"])
for i in range(0,1000):
    #j = random.randint(0,2000)
    train_ds.addSample(X_train[i], y_train[i])
    
test_ds = ClassificationDataSet(1536, nb_classes=2, class_labels=["Illegal, Legal"])
for i in range(0,1000):
    test_ds.addSample(X_test[i], y_test[i])

#print(train_ds.getField('target'))
train_ds._convertToOneOfMany(bounds=[0, 1])
#print(train_ds.getField('target'))

net     = buildNetwork(1536, 768, 2, bias=True, hiddenclass=TanhLayer)
trainer = BackpropTrainer(net,  train_ds)

for i in range(1):
    trainer.trainEpochs( 1 )
    prob_out = net.activateOnDataset(train_ds)
    out_max_prob = prob_out.argmax(axis=1)  # the highest output activation gives the class
    #out = out.reshape(X_train.shape)
    print(out_max_prob)
    print(prob_out)
#    print(confusion_matrix(y_train, out))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [33]:
prob_out

array([[ nan,  nan],
       [ nan,  nan],
       [ nan,  nan],
       ..., 
       [ nan,  nan],
       [ nan,  nan],
       [ nan,  nan]])